In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import urllib.parse

base_url = 'https://www.thesecurityevent.co.uk/'

url = base_url + 'exhibitor-list?&sortby=Community_Featured%20desc%2Ctitle%20asc&categories=0A4FB57-1CA8-4D17-809A-4EF50F2AFFFE&categories=2EE5288-2CA7-4B41-82DA-3124A0AEA9F8&searchgroup=091BE406-exhibitors'

response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

exhibitor_items = soup.find_all('li', class_='m-exhibitors-list__items__item')

with open('exhibitors.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Exhibitor', 'Stand', 'Address'])

    for exhibitor in exhibitor_items:
        exhibitor_name = exhibitor.find('h2', class_='m-exhibitors-list__items__item__header__title').text.strip()
        stand_info = exhibitor.find('div', class_='m-exhibitors-list__items__item__header__meta__stand').text.strip()
        stand = stand_info.split(':')[-1].strip()
        
        exhibitor_relative_url = exhibitor.find('a', class_='m-exhibitors-list__items__item__header__title__link')['href']
        
        exhibitor_absolute_url = urllib.parse.urljoin(base_url, exhibitor_relative_url)
        
        exhibitor_response = requests.get(exhibitor_absolute_url)
        exhibitor_soup = BeautifulSoup(exhibitor_response.content, 'html.parser')
        
        address_elem = exhibitor_soup.find('div', class_='m-exhibitor-entry__item__body__contacts__address')
        if address_elem:
            address = address_elem.get_text(separator='\n').strip()
        else:
            address = 'Address not found'
        
        writer.writerow([exhibitor_name, stand, address])
